# Visualisation routes from Airport Charles de Gaulle CDG Paris France with Folium

In [1]:
import pandas as pd
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Folium
import folium

# Get the data
## Airlines

In [2]:
df_airlines = pd.read_csv("data/airlines.dat", names=['id','name','alias','iata','icao','callsign','country','active'])
df_airlines.head(3)

,id,name,alias,iata,icao,callsign,country,active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N


## Airport data

In [3]:
# https://openflights.org/data.html#airport
df_airports = pd.read_csv("data/airports.dat", header= None, dtype=str)
df_airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude", "timezone", "dst", "databasetimezone","type ","source "]
df_airports.head(3)

,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,databasetimezone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20707988739,145.789001465,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789855957031,144.29600524902344,5388,10,U,Pacific/Port_Moresby,airport,OurAirports


## Routes data : 

In [5]:
# https://openflights.org/data.html#route
df_routes = pd.read_csv("data/routes.dat", header= None, dtype=str)
df_routes.columns = ['airline','airline_id','source','source_id','dest','dest_id','codeshare','stops','equipments']
df_routes = df_routes[df_routes["airline_id"] != "\\N"]
print(len(df_routes))
df_routes = df_routes.fillna(0)
df_routes.head(3)

67184


,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipments
0,2B,410,AER,2965,KZN,2990,0,0,CR2
1,2B,410,ASF,2966,KZN,2990,0,0,CR2
2,2B,410,ASF,2966,MRV,2962,0,0,CR2


# Routes from CDG

In [6]:
df_cdg = df_routes.loc[df_routes['source'] == 'CDG']
print(len(df_cdg))
df_cdg.head(3)

524


,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipments
1265,4U,2548,CDG,1382,HAM,342,0,0,319 CRJ
3245,8Q,3788,CDG,1382,IST,1701,0,0,320
4317,A3,96,CDG,1382,ATH,3941,0,0,321 320


In [13]:
df_cdg = df_cdg.drop(["codeshare", "stops", "equipments"], axis=1)
df_cdg = df_cdg.assign(source_lat=df_cdg.source_id)
df_cdg = df_cdg.assign(source_long=df_cdg.source_id)
df_cdg = df_cdg.assign(dest_lat=df_cdg.dest_id)
df_cdg = df_cdg.assign(dest_long=df_cdg.dest_id)
df_cdg = df_cdg.assign(airline_name=df_cdg.airline)
df_cdg = df_cdg.assign(color_airline = None)
df_cdg.head(3)

,airline,airline_id,source,source_id,dest,dest_id,source_lat,source_long,dest_lat,dest_long,airline_name,color_airline
1265,4U,2548,CDG,1382,HAM,342,1382,1382,342,342,4U,None
3245,8Q,3788,CDG,1382,IST,1701,1382,1382,1701,1701,8Q,None
4317,A3,96,CDG,1382,ATH,3941,1382,1382,3941,3941,A3,None


In [14]:
lat = df_airports.set_index('id')['latitude']
df_cdg["source_lat"] = df_cdg["source_lat"].replace(lat) 
long = df_airports.set_index('id')['longitude']
df_cdg["source_long"] = df_cdg["source_long"].replace(long) 

latd = df_airports.set_index('id')['latitude']
df_cdg["dest_lat"] = df_cdg["dest_lat"].replace(latd) 

longd = df_airports.set_index('id')['longitude']
df_cdg["dest_long"] = df_cdg["dest_long"].replace(longd)

airl = df_airlines.set_index('iata')['name']
df_cdg['airline_name'] = df_cdg['airline_name'].replace(airl)

df_cdg.head(3)

,airline,airline_id,source,source_id,dest,dest_id,source_lat,source_long,dest_lat,dest_long,airline_name,color_airline
1265,4U,2548,CDG,1382,HAM,342,49.012798,2.55,53.630401611328,9.9882297515869,Germanwings,None
3245,8Q,3788,CDG,1382,IST,1701,49.012798,2.55,40.976898,28.8146,Maldivian Air Taxi,None
4317,A3,96,CDG,1382,ATH,3941,49.012798,2.55,37.9364013672,23.9444999695,Aegean Airlines,None


In [19]:
m = folium.Map(location=[48.86, 2.33])
colour = [ "red", "blue", "green", "yellow", "purple", "orange", "white", "black" ]

for index,row  in df_cdg.iterrows():
        try:
            dest = df_airports[df_airports["id"] == row["dest_id"]].iloc[0]
            source = df_airports[df_airports["id"] == row["source_id"]].iloc[0]
            folium.PolyLine(locations = [(float(row.source_lat), float(row.source_long)),(float(row.dest_lat), 
                        float(row.dest_long))],weight=1, color='blue').add_to(m) 
            folium.Marker((float(row.dest_lat), float(row.dest_long)), popup=dest['name']).add_to(m)
            folium.Marker((float(row.source_lat), float(row.source_long)), popup=source['name']).add_to(m)

        except:
            pass


In [20]:
m

![title](images/routes_cdg.png)